In [141]:
import os
import requests
from zipfile import ZipFile
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import joblib
from tensorflow.keras.models import load_model


In [142]:
# Download dataset
url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
dataset_path = "cats_and_dogs.zip"

if not os.path.exists("dataset"):
    print("Downloading dataset...")
    response = requests.get(url)
    with open(dataset_path, 'wb') as file:
        file.write(response.content)

In [143]:
    # Extract dataset
    with ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall("dataset")


In [144]:
# Preprocess images
def preprocess_image(image_path, size=(16, 16)):  # Reduced size for faster processing
    try:
        image = cv2.imread(image_path)
        image = cv2.resize(image, size)
        image = image / 255.0  # Normalize
        return image
    except:
        return None


In [145]:
def load_data(data_dir, label_map, subset_size=None):
    images, labels = [], []
    for label, folder in label_map.items():
        folder_path = os.path.join(data_dir, folder)
        for i, filename in enumerate(os.listdir(folder_path)):
            if subset_size and i >= subset_size:
                break
            file_path = os.path.join(folder_path, filename)
            image = preprocess_image(file_path)
            if image is not None:
                images.append(image)
                labels.append(label)
    return np.array(images), np.array(labels)


In [146]:
# Load data
data_dir = "dataset/PetImages"
label_map = {0: "Cat", 1: "Dog"}
subset_size = 5000  # Use a subset for faster training
images, labels = load_data(data_dir, label_map, subset_size=subset_size)


In [147]:
# Flatten images for ML models (non-CNN models)
flattened_images = images.reshape(len(images), -1)

In [148]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
y_categorical = to_categorical(encoded_labels)


In [149]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(flattened_images, encoded_labels, test_size=0.2, random_state=42)
cnn_X_train, cnn_X_test, cnn_y_train, cnn_y_test = train_test_split(images, y_categorical, test_size=0.2, random_state=42)


In [150]:
# Train SVM
print("Training SVM...")
svm_model = SVC(kernel='linear', C=0.1, probability=True)
svm_model.fit(X_train, y_train)
joblib.dump(svm_model, "svm_model.pkl")
print("SVM training completed and saved.")


Training SVM...
SVM training completed and saved.


In [151]:
# Train Random Forest
print("Training Random Forest...")
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)
joblib.dump(rf_model, "rf_model.pkl")
print("Random Forest training completed and saved.")


Training Random Forest...
Random Forest training completed and saved.


In [152]:
# Train Logistic Regression (SGD)
print("Training Logistic Regression...")
sgd_model = SGDClassifier(loss='log_loss', max_iter=1000, random_state=42)  # Updated loss parameter
sgd_model.fit(X_train, y_train)
joblib.dump(sgd_model, "sgd_model.pkl")
print("Logistic Regression training completed and saved.")

Training Logistic Regression...
Logistic Regression training completed and saved.


In [153]:
# Train CNN
print("Training CNN...")
cnn_model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(16, 16, 3)),  # Fewer filters
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),  # Smaller dense layer
    Dense(2, activation='softmax')
])


Training CNN...


In [154]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(cnn_X_train, cnn_y_train, epochs=30, batch_size=64, validation_data=(cnn_X_test, cnn_y_test))  # Fewer epochs
cnn_model.save("cnn_model.h5")
print("CNN training completed and saved.")



Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5327 - loss: 0.7079 - val_accuracy: 0.5767 - val_loss: 0.6635
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6457 - loss: 0.6325 - val_accuracy: 0.6147 - val_loss: 0.6540
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6695 - loss: 0.6044 - val_accuracy: 0.6879 - val_loss: 0.5903
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.7077 - loss: 0.5732 - val_accuracy: 0.6889 - val_loss: 0.5803
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7181 - loss: 0.5581 - val_accuracy: 0.7069 - val_loss: 0.5681
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7329 - loss: 0.5339 - val_accuracy: 0.7014 - val_loss: 0.5702
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7481 - loss: 0.5120 - val_accuracy: 0.6979 - val_loss: 0.5727
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7510 - loss: 0.5088 - val_accu

CNN training completed and saved.


In [155]:
# Load models for inference
print("Loading models for inference...")
svm_model = joblib.load("svm_model.pkl")
rf_model = joblib.load("rf_model.pkl")
sgd_model = joblib.load("sgd_model.pkl")
cnn_model = load_model("cnn_model.h5")



Loading models for inference...


In [156]:
# Test on one sample image
sample_image = X_test[0].reshape(1, -1)  # For non-CNN models
cnn_sample_image = cnn_X_test[0].reshape(1, 16, 16, 3)  # For CNN

print("SVM Prediction:", label_encoder.inverse_transform(svm_model.predict(sample_image)))
print("Random Forest Prediction:", label_encoder.inverse_transform(rf_model.predict(sample_image)))
print("Logistic Regression Prediction:", label_encoder.inverse_transform(sgd_model.predict(sample_image)))
print("CNN Prediction:", label_encoder.inverse_transform(np.argmax(cnn_model.predict(cnn_sample_image), axis=1)))


SVM Prediction: [1]
Random Forest Prediction: [1]
Logistic Regression Prediction: [1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
CNN Prediction: [1]


In [157]:
# Train K-Means
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for clean outpu
print("Training K-Means...")
kmeans_model = KMeans(n_clusters=2, random_state=42)
kmeans_model.fit(X_train)  # Unsupervised training on flattened images
joblib.dump(kmeans_model, "kmeans_model.pkl")
print("K-Means training completed and saved.")


Training K-Means...
K-Means training completed and saved.


In [158]:
!pip install flask-ngrok flask tensorflow scikit-learn pillow

In [159]:
!pip install jupyter-dash

In [160]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data



In [161]:
pip install pyngrok


In [162]:
from flask import Flask
from pyngrok import ngrok


In [163]:
ngrok.set_auth_token('2rtA0iFOshOyTXXDlIzIHBwedUK_2PCaFQ7BbWdGDScjzNtyo')
public_url = ngrok.connect(5000).public_url
print(public_url)



https://27b5-34-19-64-63.ngrok-free.app


In [164]:
from flask import Flask, request, jsonify, render_template
from tensorflow.keras.models import load_model
import joblib
import cv2
import numpy as np
from pyngrok import ngrok



In [165]:
# Initialize Flask app
app = Flask(__name__)



In [166]:
!pkill ngrok


In [167]:
# Set up ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")



Public URL: NgrokTunnel: "https://7de7-34-19-64-63.ngrok-free.app" -> "http://localhost:5000"


In [168]:
# Load models
svm_model = joblib.load("svm_model.pkl")
rf_model = joblib.load("rf_model.pkl")
sgd_model = joblib.load("sgd_model.pkl")
cnn_model = load_model("cnn_model.h5")
kmeans_model = joblib.load("kmeans_model.pkl")  # Load KMeans model


In [169]:
# Label map
label_map = {0: "Cat", 1: "Dog"}

def inverse_label(label_idx):
    return label_map[label_idx]


In [170]:
# Preprocess image
def preprocess_image(image_file, size=(16, 16)):
    image = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_COLOR)
    if image is None:
        return None
    image = cv2.resize(image, size)
    image = image / 255.0  # Normalize
    return image



In [171]:
# Root route
@app.route('/')
def home():
    return """
    <html>
        <head><title>Cat and Dog Classifier</title></head>
        <body>
            <h1>Welcome to the Cat and  Classifier</h1>
            <form action="/predict" method="post" enctype="multipart/form-data">
                <label for="image">Upload an image:</label>
                <input type="file" name="image" accept="image/*" required>
                <button type="submit">Predict</button>
            </form>
        </body>
    </html>
    """


In [172]:
# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'}), 400

    image_file = request.files['image']
    image = preprocess_image(image_file)

    if image is None:
        return jsonify({'error': 'Invalid image format'}), 400


In [173]:
flattened_image = image.reshape(1, -1)

NameError: name 'image' is not defined

In [ ]:
# CNN requires a 4D tensor
cnn_image = image.reshape(1, 16, 16, 3)



In [ ]:
# Make predictions
    svm_prediction = inverse_label(svm_model.predict(flattened_image)[0])
    rf_prediction = inverse_label(rf_model.predict(flattened_image)[0])
    sgd_prediction = inverse_label(sgd_model.predict(flattened_image)[0])
    cnn_prediction = inverse_label(np.argmax(cnn_model.predict(cnn_image), axis=1)[0])



In [ ]:
    # KMeans prediction (returns cluster number)
    kmeans_cluster = kmeans_model.predict(flattened_image)[0]
    kmeans_prediction = f"Cluster {kmeans_cluster}"

    return jsonify({
        'svm_prediction': svm_prediction,
        'rf_prediction': rf_prediction,
        'sgd_prediction': sgd_prediction,
        'cnn_prediction': cnn_prediction,
        'kmeans_prediction': kmeans_prediction
    })



In [ ]:
if __name__ == '__main__':
app.run(port=5000)
